In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/radek1/additional-train-data-for-llm-science-exam")
od.download("https://www.kaggle.com/datasets/verracodeguacas/huggingfacedebertav3variants")
od.download("https://www.kaggle.com/datasets/datafan07/llm-whls")
od.download("https://www.kaggle.com/datasets/datafan07/rlhf-data-for-llm-science-exam")

Skipping, found downloaded files in "./additional-train-data-for-llm-science-exam" (use force=True to force download)
Skipping, found downloaded files in "./huggingfacedebertav3variants" (use force=True to force download)
Skipping, found downloaded files in "./llm-whls" (use force=True to force download)
Skipping, found downloaded files in "./rlhf-data-for-llm-science-exam" (use force=True to force download)


In [3]:
!pip install --no-index --no-deps /content/llm-whls/transformers-4.31.0-py3-none-any.whl
!pip install --no-index --no-deps /content/llm-whls/peft-0.4.0-py3-none-any.whl
#!pip install --no-index --no-deps /content/llm-whls/datasets-2.14.3-py3-none-any.whl
!pip install --no-index --no-deps /content/llm-whls/trl-0.5.0-py3-none-any.whl

Processing ./llm-whls/transformers-4.31.0-py3-none-any.whl
transformers is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing ./llm-whls/peft-0.4.0-py3-none-any.whl
peft is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing ./llm-whls/trl-0.5.0-py3-none-any.whl
trl is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [4]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [5]:
%shell
!pip install --force-reinstall datasets

  Using cached datasets-3.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached aiohttp-3.10.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.6 kB)
  Using cached huggingface_hub-0.25.1-py3

In [2]:
%shell
!pip install -U tokenizers==0.13.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1


In [8]:
!pip install numpy==1.23.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 103.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.1
    Uninstalling numpy-2.1.1:
      Successfully uninstalled numpy-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.16 requires numpy>=1.24, but you have numpy 1.23.0 which is incompatible.
albumentations 1.4.15 requires numpy>=1.24.4, but you have numpy 1.23.0 which is incompatible.
bigframes 1.18.0 requires numpy>=1.24.0, but you have numpy 1.23.0 which is incompatible.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.0 which is incompatible.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 9.2.0 requires numpy<3

In [1]:
import torch
import numpy as np
from dataclasses import dataclass, field
from typing import Optional

from datasets import load_dataset, Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, PeftModel, PeftConfig, TaskType, PeftModelForSequenceClassification
from tqdm.auto import tqdm
from transformers import (
    AutoModelForSequenceClassification, AutoModelForCausalLM, AutoModel,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    DataCollatorWithPadding
)

import random
import os
import gc



from trl import RewardTrainer


import pandas as pd
pd.set_option('display.max_colwidth', None)
tqdm.pandas()

In [2]:
class CFG:
    base_model = "/content/huggingfacedebertav3variants/deberta-v3-large"
    seed = 42

In [3]:
def set_seed(seed = int):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state


random_state = set_seed(CFG.seed)

In [4]:
train_df_0 = pd.read_csv('/content/additional-train-data-for-llm-science-exam/6000_train_examples.csv')
train_df_1 = pd.read_csv('/content/additional-train-data-for-llm-science-exam/extra_train_set.csv')
test_df = pd.read_csv('/content/Dataset/train.csv')

train_df = pd.concat((train_df_0, train_df_1), axis=0)
train_df.dropna(inplace=True)
train_df.reset_index(drop=True, inplace=True)

In [5]:
def generate_new_dataframe(df):
    new_rows = []


    for _, row in df.iterrows():
        prompt = row['prompt']
        chosen_option = row[row['answer']]

        for option in ['A', 'B', 'C', 'D', 'E']:
            if option != row['answer']:
                rejected_option = row[option]
                new_row = {'chosen': prompt + ' ' + chosen_option, 'rejected': prompt + ' ' + rejected_option}
                new_rows.append(new_row)

    new_df = pd.DataFrame(new_rows)
    return new_df


train_df = generate_new_dataframe(train_df)
test_df = generate_new_dataframe(test_df)

In [6]:
train_df_2 = pd.read_csv('/content/rlhf-data-for-llm-science-exam/llm_rlhf_extra.csv')
train_df = pd.concat((train_df, train_df_2), axis=0)
train_df = train_df.sample(frac=1.0, random_state=CFG.seed)

In [7]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    CFG.base_model,
    num_labels=1,

)

tokenizer = AutoTokenizer.from_pretrained(CFG.base_model)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:460: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

In [9]:
peft_config = LoraConfig(
    r=8, lora_alpha=4, task_type=TaskType.SEQ_CLS, lora_dropout=0.1,
    bias="none", inference_mode=False, target_modules=["query_proj", "value_proj"]
)

In [10]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 788,482 || all params: 435,850,242 || trainable%: 0.1809066335220711


In [11]:
def preprocess_function(examples):
    new_examples = {
        "input_ids_chosen": [],
        "attention_mask_chosen": [],
        "input_ids_rejected": [],
        "attention_mask_rejected": [],
    }
    for chosen, rejected in zip(examples["chosen"], examples["rejected"]):
        tokenized_j = tokenizer(chosen, truncation=True)
        tokenized_k = tokenizer(rejected, truncation=True)

        new_examples["input_ids_chosen"].append(tokenized_j["input_ids"])
        new_examples["attention_mask_chosen"].append(tokenized_j["attention_mask"])
        new_examples["input_ids_rejected"].append(tokenized_k["input_ids"])
        new_examples["attention_mask_rejected"].append(tokenized_k["attention_mask"])

    return new_examples

train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
)
train_dataset = train_dataset.filter(
    lambda x: len(x["input_ids_chosen"]) <= 256
    and len(x["input_ids_rejected"]) <= 256
)


test_dataset = test_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
)
test_dataset = test_dataset.filter(
    lambda x: len(x["input_ids_chosen"]) <= 2048
    and len(x["input_ids_rejected"]) <= 2048
)

Map (num_proc=4):   0%|          | 0/30116 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Filter:   0%|          | 0/30116 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/800 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Filter:   0%|          | 0/800 [00:00<?, ? examples/s]

In [12]:
training_args = TrainingArguments(
    output_dir='op',
    overwrite_output_dir = True,
    warmup_ratio=0.1,
    lr_scheduler_type='cosine',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    remove_unused_columns=False,
    optim="adafactor",
    logging_steps=250,
    eval_steps=250,
    evaluation_strategy='steps',
    load_best_model_at_end=True,
    save_total_limit = 2,
    fp16=True,
    bf16=False,
    weight_decay=0.01,
    report_to="none",
)

In [13]:
trainer = RewardTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    max_length=256,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [14]:
model.config.use_cache = False
trainer.train()
trainer.save_model('deberta_adapter')

del model

gc.collect()
torch.cuda.empty_cache()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2411: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Accuracy
250,0.696700,0.692341,0.717500
500,0.652600,0.527688,0.853750
750,0.555000,0.457421,0.871250
1000,0.526600,0.421653,0.862500
1250,0.508200,0.408440,0.867500
1500,0.503500,0.401011,0.866250
1750,0.495000,0.399697,0.865000


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2411: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2411: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2411: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickl

In [15]:
df = pd.read_csv('/content/Dataset/train.csv')

In [16]:
def get_score(model, tokenizer, prompt, response):
    inputs = tokenizer(prompt + ' ' + response, return_tensors="pt", max_length=2048, padding='longest', truncation=True).to('cuda')
    model.to('cuda')
    model.eval()
    with torch.autocast('cuda', dtype=torch.float16):
        outputs = model(input_ids = inputs['input_ids'], attention_mask=inputs['attention_mask'])
    logits = outputs.logits

    return logits.item()

def get_top_3_winners(model, tokenizer, prompt, response_options):
    scores = []
    for index, response in enumerate(response_options):
        score = get_score(model, tokenizer, prompt, response)
        scores.append((index, score))


    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)

    top_3_winners = sorted_scores[:3]
    top_3_winners = [t[0] for t in top_3_winners]

    int_to_string = {
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D',
    4: 'E'
    }

    top_3_winners = [int_to_string[val] for val in top_3_winners]


    return top_3_winners

In [17]:
preds = []
for _, row in tqdm(df.iterrows()):
    prompt = row['prompt']
    response_options = [
        row['A'],
        row['B'],
        row['C'],
        row['D'],
        row['E']
    ]
    top_3_winners = get_top_3_winners(trainer.model, tokenizer, prompt, response_options)
    preds.append(top_3_winners)

final_preds = [' '.join(pred) for pred in preds]

0it [00:00, ?it/s]

In [18]:
def precision_at_k(r, k):
    """Precision at k"""
    assert k <= len(r)
    assert k != 0
    return sum(int(x) for x in r[:k]) / k

def MAP_at_3(predictions, true_items):
    """Score is mean average precision at 3"""
    U = len(predictions)
    map_at_3 = 0.0
    for u in range(U):
        user_preds = predictions[u]
        user_true = true_items[u]
        user_results = [1 if item == user_true else 0 for item in user_preds]
        for k in range(min(len(user_preds), 3)):
            map_at_3 += precision_at_k(user_results, k+1) * user_results[k]
    return map_at_3 / U


MAP_at_3(final_preds, df['answer'])

0.7383333333333332

In [19]:
df = pd.read_csv('/content/Dataset/test.csv')
preds = []
for _, row in tqdm(df.iterrows()):
    prompt = row['prompt']
    response_options = [
        row['A'],
        row['B'],
        row['C'],
        row['D'],
        row['E']
    ]
    top_3_winners = get_top_3_winners(trainer.model, tokenizer, prompt, response_options)
    preds.append(top_3_winners)
final_preds = [' '.join(pred) for pred in preds]

0it [00:00, ?it/s]

In [20]:
sub = pd.read_csv('/content/Dataset/sample_submission.csv')
sub['prediction'] = final_preds

In [21]:
sub.to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head()

,id,prediction
0,0,D B E
1,1,A B C
2,2,A C E
3,3,C E A
4,4,D A B
